In [3]:
import scipy as sp
import numpy as np
import os
import matplotlib.pyplot as plt
from scipy import signal
from scipy.signal import argrelextrema
from scipy import fftpack
import math

In [4]:
csv_dir_path = "../data/Part_1"

In [5]:
csv_paths = []

for patNum in range(1,16):
    csv_paths.append(os.path.join(csv_dir_path, str(patNum)+".csv"))

for i in range(15):
    Y = np.genfromtxt(csv_paths[i], delimiter=",")
    num_rows, num_cols = Y.shape
   
    for k in range(0, num_cols, 1000):
        if(num_cols-k >= 2000):
            # Extracting row corresponding to PPG signal
            # Note that the first row of csv file is PPG signal
            PPG = Y[0, k:k+1000]

            # Extracting row corresponding to BP signal
            BP = Y[1, k:k+1000]

            # Extracting row corresponding to ECG signal
            ECG = Y[2, k:k+1000]
        else:
            PPG = Y[0, k:]
            BP = Y[1, k:]
            ECG = Y[2, k:]

        # First derivative of PPG signal
        PPG1 = np.gradient(PPG)

        # figure('PPG 1st derivative')
        # Uncomment below to check the graphs found from the first derivative of PPG
        """
        plt.plot(range(len(PPG_diff)),PPG1)
        plt.xlabel("PPG first derivative")
        plt.show()
        """
        # Second derivative of PPG signal
        PPG2 = np.gradient(PPG1)

        # figure('PPG 2nd derivative')
        # Uncomment below to check the graphs found from the second derivative of PPG
        """
        plt.plot(range(len(PPG2)),PPG2)
        plt.xlabel("PPG second derivative")
        plt.show()
        """
        #! Need of these 4 lines of code?
        F = np.ones(1000)
        np.append(F, PPG)
        np.append(F, PPG2)
        np.append(F, ECG)

        Len_PPG2 = len(PPG2)

        # Sampling frequency = 125 Hz
        Fs = 125
        # Time vector based on sampling rate
        Ts = 1 / Fs  # 0.008

        T = np.arange(0, 8, 0.008)

        # Find peaks or local maximum of PPG signals
        [PPG_max_pk, loc] = signal.find_peaks(PPG)
        # Finding the local minimum of the PPG signals
        #! https://stackoverflow.com/questions/4624970/finding-local-maxima-minima-with-numpy-in-a-1d-numpy-array (easier one maybe)
        PPG_diff = max(PPG) - PPG
        [PPG_min_pk, loc1] = signal.find_peaks(PPG_diff)

        # Systolic time
        sys_time = 0

        for i in range(1, 5, 1): #changed (SID)
            sys_time = sys_time + T[PPG_max_pk[i]] - T[PPG_min_pk[i]]

        sys_time = sys_time / 5

        # Diastolic time
        dias_time = 0

        for i in range(1, 5, 1): #changed (SID)
            sys_time = sys_time + T[PPG_min_pk[i+1]] - T[PPG_max_pk[i]]

        dias_time = dias_time / 5
        #! What's this??
        v = [0.1, 0.25, 0.33, 0.5, 0.66, 0.75]

        ppg_21_st = []
        ppg_21_dt = []

        a = 0
        b = 0

        for j in range(1, 6, 1): #changed (SID)
            for i in range(PPG_min_pk[1], PPG_max_pk[1], 1):
                if PPG[i] >= (v[j] * PPG[1] + PPG_diff[1]): #! diff from original code
                    a = i
                    break

            for i in range(PPG_max_pk[1], PPG_min_pk[2], 1):
                if PPG[i] <= (v[j] * PPG[1] + PPG_diff[1]):
                    b = i
                    break

            ppg_21_st.append((PPG_max_pk[1] - a) * 0.008)
            ppg_21_dt.append((b - PPG_max_pk[1]) * 0.008)

        # max value of ECG signal
        [ECG_max_pk, loc2] = signal.find_peaks(ECG, prominence=(None, 0.6))

        [PPG2_max_pk, loc3] = signal.find_peaks(PPG2, prominence=(None, 0.003))

        n = len(ECG_max_pk)  # to find out vector dimensions of ECG signal
        y = len(PPG2_max_pk)

        P1 = np.take(T, ECG_max_pk)
        P = np.take(T, PPG2_max_pk)
        P11 = P1[1:n]
        P2 = P[1:y]
        ptt = 0

        temp = min(y, n)
        minRange = min(temp, 5)

        for i in range(minRange - 1):
            ptt = ptt + abs(P2[i] - P11[i])

        ptt = ptt / minRange
        # print(ptt)

        lr1 = len(PPG_min_pk)
        # print(lr1)
        rationum = 0
        ratioden = 0

        ih = 0
        il = 0

        for i in range(1, lr1 - 1, 1):
            rationum = rationum + PPG_max_pk[i]
            ratioden = ratioden + PPG_min_pk[i]

        ih = rationum / (lr1 - 1)
        il = ratioden / (lr1 - 1)
        # print(il,' il,ih ',ih)

        PIR = ih / il
        RR = fftpack.diff(P1)  # to find time taken for 1 heartbeat

        HR = np.zeros(len(RR)).astype(np.float32)
        HR = np.divide(60, RR, HR)
        # print(HR)

        # hrfinal = np.zeros(len(HR)).astype(np.float32)
        hrfinal = 0
        lr1 = len(HR)
        tlr1 = lr1

        for i in range(1, lr1, 1):
            t = HR[i]

            if t <= 30 or t >= 200:
                tlr1 = tlr1 - 1
            else:
                hrfinal = hrfinal + HR[i]

        hrfinal = hrfinal / (tlr1);    
        #print(hrfinal)
        
        '''
        plt.plot(T,P01)
        plt.xlabel("T")
        plt.ylabel("PPG signal")
        plt.show()
        '''
        
        '''
        plt.plot(T,E01)
        plt.xlabel("T")
        plt.ylabel("ECG signal")
        plt.show()
        '''
        Yy = np.fft.fft(PPG)
        # print(Yy[0])
        Yy[0] = 0
        S  = np.real(np.fft.ifft(Yy))

        [pk4, loc4] = signal.find_peaks(S)
        [pk5, loc5] = signal.find_peaks(BP)

        lr1 = len(pk4)
        iftmax = 0

        for i in PPG_max_pk:
            iftmax += S[i]

        meu = iftmax/(lr1-1)

        # print(il, meu, hrfinal)
        alpha = il * math.sqrt(1060 * hrfinal / meu)

        BP1 = max(BP) - BP
        [pk6, loc6] = signal.find_peaks(BP1)

        lr1 = len(pk5)
        bpmax = 0

        for i in PPG_max_pk:
            bpmax += BP[i]

        bpmax /= (lr1-1)
        print(bpmax)

        lr1 = len(pk6)
        bpmin = 0

        for i in pk6:
            bpmin += BP1[i]

        bpmin /= (lr1-1)
        print(bpmin)

74.243
62.59680000000001
85.33066666666666
62.28153333333334
75.84335294117648
63.863529411764695
32.98974358974359
60.39597499999998
34.668171428571426
61.87205714285714
84.10123529411764
64.10047058823528
87.0662
65.185
75.91881249999999
63.92631249999998
80.17593333333333
66.20213333333332
75.9006875
65.4115625
80.29668750000002
63.0739375
87.91286666666664
64.5706875
75.7664375
65.48280000000001
86.34660000000001
65.32112500000001
79.85393750000001
64.52473333333333
61.18945
63.73895
85.76693333333334
65.93081249999997
76.60287499999998
66.36760000000002
78.23606250000002
65.453
94.5754
67.2643125
85.14437500000001
65.27687500000002
71.02957894736842
64.87326315789475
86.22794736842107
67.13663157894737
40.3732
58.08474285714285
54.89189285714286
66.71650000000001
57.49635714285715
66.13775
49.97038235294117
67.6910588235294
47.77525714285714
63.968599999999995
30.512019999999996
80.36320000000002
70.45554545454546
66.76877272727272
85.01470588235294
66.57182352941176
75.56915
66.5